In [7]:
import os, math, numpy as np
import pandas as pd
import copy
from collections import defaultdict

In [6]:
def get_distance(source: tuple, destination : tuple) -> float:
    return ((source[1] - destination[1]) ** 2 + (source[0] - destination[0]) ** 2)**0.5

In [7]:
### Calculate travel time between two points: origin and destination
def traveltime(origin_id,destination_id,meters_per_minute,locations):
    dist=np.sqrt((locations.at[destination_id,'x']-locations.at[origin_id,'x'])**2\
                +(locations.at[destination_id,'y']-locations.at[origin_id,'y'])**2)
    tt=np.ceil(dist/meters_per_minute)
    return tt

# Read in data instance
A folder of data instance has 5 files: (1) couriers.txt, (2) instance_characteristics.txt, (3) instance_parameters.txt, (4) orders.txt, (5) restaurants.txt

In [3]:
# Read instance information
def read_instance_information(instance_dir):
    orders=pd.read_table(os.path.join(instance_dir,'orders.txt'))
    restaurants=pd.read_table(os.path.join(instance_dir,'restaurants.txt'))
    couriers=pd.read_table(os.path.join(instance_dir,'couriers.txt'))
    instanceparams=pd.read_table(os.path.join(instance_dir,'instance_parameters.txt'))

    order_locations=pd.DataFrame(data=[orders.order,orders.x,orders.y]).transpose()
    order_locations.columns=['id','x','y']
    restaurant_locations=pd.DataFrame(data=[restaurants.restaurant,restaurants.x,restaurants.y]).transpose()
    restaurant_locations.columns=['id','x','y']
    courier_locations=pd.DataFrame(data=[couriers.courier,couriers.x,couriers.y]).transpose()
    courier_locations.columns=['id','x','y']
    locations=pd.concat([order_locations,restaurant_locations,courier_locations])

    meters_per_minute=instanceparams.at[0,'meters_per_minute']
    pickup_service_minutes=instanceparams.at[0,'pickup service minutes']
    dropoff_service_minutes=instanceparams.at[0,'dropoff service minutes']
    target_click_to_door=instanceparams.at[0,'target click-to-door']
    pay_per_order=instanceparams.at[0,'pay per order']
    guaranteed_pay_per_hour=instanceparams.at[0,'guaranteed pay per hour']

    return orders,restaurants,couriers,instanceparams,locations

In [4]:
instance_dir = 'data/0o50t75s1p100'
orders,restaurants,couriers,instanceparams,locations=read_instance_information(instance_dir)

In [10]:
locations

,id,x,y
0,o1,8317,5587
1,o2,8183,4908
2,o3,4953,5140
3,o4,5268,3678
4,o5,6245,3071
...,...,...,...
56,c57,6324,3843
57,c58,7313,88
58,c59,9894,1883
59,c60,3073,5511


# Set hyper-parameters:
f∶ every f minutes solves a matching problem to prescribe the next pick-up and delivery assignment for each courier<br>
t : optimization time<br>
∆_(u): t+ ∆_(u ) is the assignment horizon<br>
∆_(1); ∆_(2): to determine Z_(t)<br>
beta: control the freshness in the construction of bundles
gamma: control the click to door time in the construction of bundles


In [10]:
# Accorting to default values in the paper
f = 5
delta_u = 10
delta_1 = 10
delta_2 = 10
# Not find in the paper
beta = 10 # should be tuned
gamma = 10 # should be tuned

# Procedure to initialize a solution

**Function that return U(t,r)**<br>
U(t,r) is the set of upcoming orders at restaurant r

In [11]:
# Done
def create_set_U_tr(t,r):
    pass
# Done under class DeliverRouting

**Order Object**

In [12]:
class Order:
    def __init__(self, order_information : dict):

        self.id = order_information.get('order')
        self.destination = (order_information.get('x'), order_information.get('y'))
        self.placement_time = order_information.get('placement_time')
        self.restaurant_id = order_information.get('restaurant')
        self.ready_time = order_information.get('ready_time')
        
        # To be updated after assignment
        self.pickup_time = 0
        self.dropoff_time = 0
        self.courier_id = ""

**Route object**<br>
A route is a bundle,i.e: a list of ordered orders

In [9]:
class Route(object):
    def __init__(self,bundle : list, restaurant_id : str):
        self.bundle = bundle
        self.ready_time = 0
        self.restaurant_id = restaurant_id
        
    def get_ready_time(self):
        self.ready_time = max([o.ready_time for o in self.bundle])

    # calculate total travel time from 1st destination to the last destination of the route
    def total_travel_time():
        pass

    # calculate total service delay    
    def total_service_delay():
        #<to be discuss>
        pass

    # calculate total_click_to_door 
    def total_click_to_door():
        #<to be discuss>
        pass

    # calculate route efficiency: travel time per order:
    def route_efficiency():
        pass

    # calculate route cost
    def route_cost():
        #route_cost = total_travel_time + beta*total_service_delay + gamma*total_click_to_door
        pass


**Assignment Object**

In [ ]:
class Assignment():
    def __init__(self, assign_time : int , restaurant_id : str, courier: object, route : Route):
        self.assign_time = assign_time
        self.restaurant_id = restaurant_id  # each assignment of a bundle has only one corresponding restaurant
        self.pickup_time = 0
        self.courier = courier
        self.route = route
        self.isfinal_flag = 0 # indicate if the assignment is final (can not be updated)
        self.ready_time = route.ready_time

**Courier Object**

In [13]:
class Courier(object):
    def __init__(self, courier_information : dict):
        self.id = courier_information.get('courier')
        self.x = courier_information.get('x')
        self.y = courier_information.get('y')
        self.on_time = courier_information.get('on_time')
        self.off_time = courier_information.get('off_time')

        # Sequence of moves of the courier
        # Update and derived along the way
        self.assignment = []                           # containing assigned assigments 
        self.next_available_time = 0                   # when the courier is available for the next assignment (or the drop off time of the last order of the last assignment)
        self.position_after_last_assignment = self.id  # the position of the courier after completing the last assignment

   # check if the courier can take a bundle. 
    def can_assign(self, route : Route) -> bool:
        if route.ready_time < self.on_time:
            return False
        if route.ready_time > self.off_time:
            return False

    # assign a bundle to the courier
    def assign_bundle(self, route: Route):
        # calculate courier's arrival time to the bundle's restaurant:
        arrival_time = self.next_available_time +\
                        

        



        
        

            

In [17]:
class DeliveryRouting:
    def __init__(self, instance_dir : str, target_bundle_size : int):
        self.target_bundle_size = target_bundle_size

        orders, restaurants, couriers, instanceparams, locations = read_instance_information(instance_dir)

        self.orders = [Order(order) for order in orders.to_dict(orient = 'records')]
        self.orders = sorted(self.orders, key = lambda x: x.order_id)

        self.restaurants = restaurants.to_dict(orient = 'records')
        self.couriers = [Courier(courier) for courier in couriers.to_dict(orient = 'records')]
        self.unassigned_orders = self.copy(self.orders)

        self.instance_params = instanceparams.to_dict(orient = 'records')[0]

        self.meters_per_minute, self.pickup_service_minutes, self.dropoff_service_minutes, \
            self.target_click_to_door, self.maximum_click_to_door, self.pay_per_order,\
            self.guaranteed_pay_per_hour = self.instance_params.values()
        
        self.orders_by_horizon_interval = defaultdict(list)
        self.locations = locations

    def traveltime(self, origin_id,destination_id):
        dist=np.sqrt((self.locations.at[destination_id,'x']-self.locations.at[origin_id,'x'])**2\
                    +(self.locations.at[destination_id,'y']-self.locations.at[origin_id,'y'])**2)
        tt=np.ceil(dist/self.meters_per_minute)
        return tt

    def get_ready_orders(self,f: int,delta_u: int) -> dict:
        '''
        This function return orders which have ready time fall into the corresponding horizon.
        This function should be run only once.
        '''
        # starting time of each interval
        f_list = [*range(0,24*60+1,f)]
        # get ready orders correspoding to each horizon interval
        for i in range(1,len(f_list)):
            for o in self.orders:
                if o.placement_time < f_list[i] and o.placement_time >= f_list[i-1] and o.ready_time < f_list[i]+delta_u:
                     self.orders_by_horizon_interval[f].append(o)

    def get_num_ready_orders(self,f):
        '''
        Function that return U(t,r)
        U(t,r) is the set of upcoming orders at restaurant r
        '''
        return len(self.orders_by_horizon_interval[f])


    
    def copy(self):
        return copy.deepcopy(self)

    def can_arrive(self,
                   source : tuple,
                   restaurant : tuple,
                   destination : tuple,
                   current_time : float,
                   ready_time : float):

        travel_time = self.calculate_travel_time(source, restaurant, destination)

        if current_time + travel_time <= ready_time:
            return False
        return True

    def get_bundle_size(self, z : int) -> int :
        # available = [courier for courier in self.couriers if courier.]
        pass

    def number_of_bundles(self, restaurant_id, t):

        # First get the time when they move from current position to each order for each restaurant


        # If arrival_time >= ready_time: count










    def random_initialization(self):
        pass

In [18]:
instance_dir = 'data/0o50t75s1p100'
dr = DeliveryRouting(instance_dir)

**Funtion that calculate bundel size Z(t)**

In [ ]:
def get_bundel_size():
    ...

**Function that calculate k(r,t)**<br>
k(r,t) is the number of couriers who are available at restaurant r

In [ ]:
def get_available_courier(t,r):
    ...

# Implement procedure 1